<a href="https://colab.research.google.com/github/COVID19Tracking/covid19-datafetcher/blob/master/tools/%5BCTP%5D_Long_Formulas_and_Cats_(CTP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CTP Publish Shift: Long-long-but-easy Formulas

### How to use:
Running a specific cell:  
click the ▶️ (play) button text to the cell, or `ctrl+Enter` the cell

Running the entire notebook:  
"Runtime" menu -> "Run all", or `ctrl-F9`


In [ ]:
# DE positives
import pandas as pd
from datetime import datetime 
print("Last Run", datetime.now())
df = pd.read_csv('https://myhealthycommunity.dhss.delaware.gov/locations/state/download_covid_19_data')
df = df[df['Unit'] == 'tests'].set_index(['Year', 'Month', 'Day']).sort_index()
df.loc[df.index.unique()[-3]][['Statistic', 'Value']]


Last Run 2021-01-21 15:31:56.306712


Statistic      Value
Year Month Day                           
2021 1     17   Positive Tests    74311.0
           17      Total Tests  1142091.0

In [ ]:
# HI PCR Test Encounters
import pandas as pd
import requests
from datetime import datetime, timezone
from pytz import timezone as tz  # replace with ZoneInfo once G upgrades to 3.9

hi = pd.read_csv("https://public.tableau.com/views/EpiCurveApr4/CSVDownload.csv?:showVizHome=no")
print(hi.select_dtypes(exclude=['object']).sum())


# HI updated time
res = requests.get("https://services9.arcgis.com/aKxrz4vDVjfUwBWJ/arcgis/rest/services/HIEMA_TEST_DATA_PUBLIC_LATEST/FeatureServer/0/query?where=name%3D'State'&returnGeometry=false&outFields=*&orderByFields=reportdt desc&resultOffset=0&resultRecordCount=1&f=json")
updated = datetime.fromtimestamp(res.json()['features'][0]['attributes']['reportdt']/1000) # because ms
# format we want: 12/27/2020 8:30:00
print("\nUpdate time: ", updated.replace(tzinfo=timezone.utc).astimezone(tz=tz("Pacific/Honolulu")).strftime("%m/%d/%Y %H:%M:%S"))


New Cases                 20742
New Positive Tests        20964
Total Test Encounters    803715
dtype: int64

Update time:  12/27/2020 08:30:00


In [ ]:
# MA
from io import StringIO, BytesIO
from bs4 import BeautifulSoup
import pandas as pd
import re
import requests
import zipfile

url = 'https://www.mass.gov/info-details/covid-19-response-reporting'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
a = soup.find('a', string=re.compile("COVID-19 Raw Data"))
link = "https://www.mass.gov{}".format(a['href'])
print("Download link = ", link)

res = requests.get(link)
tabs = pd.read_excel(res.content, sheet_name=None)

print("PCR Total People")
print(tabs['Testing2 (Report Date)']['Molecular Total'].iloc[-1], "\n")

df = tabs['TestingByDate (Test Date)'].filter(like="All Positive")
print(df.sum())

# weekly report
url = 'https://www.mass.gov/info-details/covid-19-response-reporting'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
a = soup.find('a', string=re.compile("Weekly Public Health Report - Raw"))
link = "https://www.mass.gov{}".format(a['href'])
print("\nWeekly link = ", link)
res = requests.get(link)
df = pd.read_excel(BytesIO(res.content), sheet_name='Antibody', parse_dates=['Test Date'], index_col='Test Date')
print(df.sum())

# ever hospitalized
print('\nEver Hospitalized')
max_date = tabs['RaceEthnicityLast2Weeks']['Date'].max()
tabs['RaceEthnicityLast2Weeks'][tabs['RaceEthnicityLast2Weeks']['Date'] == max_date].sum()

Download link =  https://www.mass.gov/doc/covid-19-raw-data-january-14-2021/download
PCR Total People
4287412 

All Positive Molecular Tests                         520263.0
All Positive Molecular Tests_Higher Ed ONLY            7908.0
All Positive Molecular Tests_MA without Higher ED    360938.0
dtype: float64

Weekly link =  https://www.mass.gov/doc/weekly-public-health-report-raw-data-january-14-2021/download
Positive Tests                             13369.0
Total Tests                               143396.0
Unnamed: 3                                     0.0
DATA CURRENT AS OF 11:59PM ON 01/12/21         0.0
dtype: float64

Ever Hospitalized


Race/Ethnicity        White, non-HispanicBlack or African American, ...
All Cases                                                        448447
Ever Hospitaltized                                                17109
Deaths                                                            13359
dtype: object

In [ ]:
# ME 
import pandas as pd
import requests
from io import StringIO

url = "https://gateway.maine.gov/dhhs-apps/mecdc_covid/hospital_capacity.csv"
pd.read_csv(url, nrows=1).filter(like='COVID')

,Hospitalized- Total COVID-19 Patients,Hospitalized- COVID-19 Patients In Critical Care,Hospitalized- COVID-19 Patients on Ventilators
0,193,59,20


In [ ]:
# MI Testing
import pandas as pd
import requests
from bs4 import BeautifulSoup

url = 'https://www.michigan.gov/coronavirus/0,9753,7-406-98163_98173---,00.html'

req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
a = soup.find('a', string="Diagnostic Tests by Result and County")
mi_link = "https://www.michigan.gov/{}".format(a['href'])
print("Link = ", mi_link)

mi = pd.read_excel(mi_link).drop(columns=['COUNTY'])
mi.sum()

Link =  https://www.michigan.gov//documents/coronavirus/Diagnostic_Tests_by_Result_and_County_2021-01-05_712059_7.xlsx


Negative    7772844
Positive     638069
Total       8410913
dtype: int64

In [ ]:
# NC Antigen tests
import pandas as pd
nc = pd.read_csv("https://public.tableau.com/views/NCDHHS_COVID-19_DataDownload/DailyTestingMetrics.csv", parse_dates=['Date'], index_col='Date', thousands=',')
nc.pivot(columns='Measure Names').sum().astype('int64')

                Measure Names           
Measure Values  Antigen Test                 224620
                Daily Tests Total           6465155
                Molecular Test              3409541
                Positive Test Percentage         20
dtype: int64

In [ ]:
# ND Negatives + Testing
import pandas as pd
import requests
from io import StringIO

url = "https://static.dwcdn.net/data/NVwou.csv"
headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0"}
req = requests.get(url, headers=headers)
print(pd.read_csv(StringIO(req.text)).filter(like='Negative').sum())

print("\n")
print("Testing Data")
df = pd.read_csv('https://www.health.nd.gov/sites/www/files/documents/Files/MSS/coronavirus/charts-data/PublicUseData.csv')
df.filter(like='tests').sum()

Total Negative    290753
dtype: int64


Testing Data


Total tests (new people)                          386131.0
Total tests (susceptible test encounters)        1351169.0
Total PCR tests (susceptible test encounters)    1294101.0
Total Ag tests (susceptible test encounters)       57068.0
Total tests (specimens)                          1435657.0
Total PCR tests (specimens)                      1369503.0
Total Ag tests (specimens)                         66154.0
dtype: float64

In [ ]:
# OH testing
import requests
import pandas as pd

key_url = "https://data.ohio.gov/apigateway-secure/data-portal/download-file/cba54974-06ab-4ec8-92bc-62a83b40614e?key=2b4420ffc0c5885f7cd42a963cfda0b489a9a6dff49461e1a921b355ee0424c029cf4ff2ee80c8c82ef901d818d71f9def8cba3651f6595bd6a07e1477438b97bbc5d7ccf7b5b66c154779ce7a4f5b83"
testing_url = "https://data.ohio.gov/apigateway-secure/data-portal/download-file/2ad05e55-2b1a-486c-bc07-ecb3be682d29?key=e42285cfa9a0b157b3f1bdaadcac509c44db4cfa0f90735e12b770acb1307b918cee14d5d8e4d4187eb2cab71fc9233bda8ee3eed924b8a3fad33aaa6c8915fe6f3de6f82ad4b995c2359b168ed88fa9"
url = testing_url

pd.read_csv(requests.get(url).json()['url']).filter(like='Daily').sum()

Confirmatory Lab Total Tested (Daily)                        8225635.000
Confirmatory Lab Total Positive (Daily)                       728881.000
Confirmatory Lab Percent Positive (Daily)                         25.260
Presumptive Lab Total Tested (Daily)                          993668.000
Presumptive Lab Total Positive (Daily)                         55536.000
Presumptive Lab Percent Positive (Daily)                           5.746
Confirmatory and Presumptive Lab Total Positive (Daily)       597733.000
Confirmatory and Presumptive Lab Total Tested (Daily)        5461978.000
Confirmatory and Presumptive Lab Percent Positive (Daily)          9.627
dtype: float64

In [ ]:
_# TX
import pandas as pd
import requests
from datetime import datetime, timedelta

url = 'https://www.dshs.texas.gov/coronavirus/TexasCOVID-19HospitalizationsOverTimebyTSA.xlsx'
df = pd.read_excel(url, sheet_name='COVID-19 ICU', skiprows=2)
print("ICU")
print(df.loc[df[df.columns[0]] == 'Total'][df.columns[-1]])


# PCR Positives
res = requests.get('https://services5.arcgis.com/ACaLB9ifngzawspq/arcgis/rest/services/TX_DSHS_COVID19_TestData_Service/FeatureServer/6/query?where=1%3D1&outStatistics=%5B%7B%27statisticType%27%3A+%27sum%27%2C+%27onStatisticField%27%3A+%27NewPositive%27%7D%2C+%7B%27statisticType%27%3A+%27sum%27%2C+%27onStatisticField%27%3A+%27OldPositive%27%7D%5D&f=json')
print("\nPCR Positives")
print(sum(res.json()['features'][0]['attributes'].values()))

res = requests.get('https://services5.arcgis.com/ACaLB9ifngzawspq/ArcGIS/rest/services/TX_DSHS_COVID19_Cases_Service/FeatureServer/2/query?where=1%3D1&outFields=%2A&orderByFields=Date+desc&resultRecordCount=1&f=json')
print("\nCases Timestamp (as-of)")
cases_date = datetime.fromtimestamp(res.json()['features'][0]['attributes']['Date']/1000)
# convent to TX time through trickery (from UTC)
print(cases_date - timedelta(hours=6))

# Antigen Positives
res = requests.get('https://services5.arcgis.com/ACaLB9ifngzawspq/ArcGIS/rest/services/TX_DSHS_COVID19_TestData_Service/FeatureServer/3/query?where=1%3D1&objectIds=&time=&resultType=none&outFields=*&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&sqlFormat=none&f=json')
print("\nAntigen Positives")
print(res.json()['features'][5]['attributes']['Count_'])

# Antibody Positives
print("\nAntibody Positives")
print(res.json()['features'][2]['attributes']['Count_'])

ICU
22    3352.0
Name: 2021-01-24, dtype: float64

PCR Positives
2227866

Cases Timestamp (as-of)
2021-01-25 00:00:00

Antigen Positives
154475

Antibody Positives
117468


In [ ]:
# UT 
from io import StringIO, BytesIO
import pandas as pd
import requests
import zipfile

url = 'https://coronavirus-dashboard.utah.gov/Utah_COVID19_data.zip'
res = requests.get(url)
zipdata = BytesIO(res.content)
zip = zipfile.ZipFile(zipdata, 'r')
for zf in zip.filelist:
  if zf.filename.startswith('Overview_Total Tests by Date'):
    # yay, the testing file
    title = 'Tests'
  elif zf.filename.startswith('Overview_Number of People Tested by Date'):
    title = 'People'
  else:
    title = None
  if title:
    title = "Metrics for {} (from {})".format(title, zf.filename)
    print(title, "\n"+"="*len(title))
    df = pd.read_csv(zip.open(zf.filename)).drop(columns=[' Total Daily Tests', 'Total Positive Tests', 'Daily People Tested', 'Daily Positive Tests'], errors="ignore")
    print(df.groupby(['Test Type', 'Result']).sum())

Metrics for People (from Overview_Number of People Tested by Date_2021-01-14.csv) 
                              Count
Test Type         Result           
Antigen by DFA/IF NEGATIVE   187552
                  POSITIVE    40351
PCR/amplification NEGATIVE  1366779
                  POSITIVE   276934
Metrics for Tests (from Overview_Total Tests by Date_2021-01-14.csv) 
                              Count
Test Type         Result           
Antigen by DFA/IF NEGATIVE   517096
                  POSITIVE    42040
PCR/amplification NEGATIVE  2121421
                  POSITIVE   300381


In [ ]:
# WI PCR Testing Encounters
import pandas as pd
from datetime import datetime
print("Last run at: ", datetime.now().isoformat())
wi = pd.read_csv("https://bi.wisconsin.gov/t/DHS/views/PercentPositivebyTestPersonandaComparisonandTestCapacity/TestCapacityDashboard.csv", thousands=",")
wi[wi['Measure Names'] == 'Total people tested daily']['Totals'].sum()

Last run at:  2020-11-18T19:01:53.066877


3986414